## Import Libraries

In [17]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

## Data

mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)


BUFFER_SIZE = 10000


suffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = suffled_train_and_validation_data.take(num_validation_samples)
train_data = suffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

## Model

### Outline the model

In [29]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

### Choose the optimizer and the loss function

In [30]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

In [31]:
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 28s - loss: 0.3284 - accuracy: 0.9076 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
540/540 - 23s - loss: 0.1363 - accuracy: 0.9605 - val_loss: 0.1118 - val_accuracy: 0.9678
Epoch 3/5
540/540 - 21s - loss: 0.0963 - accuracy: 0.9709 - val_loss: 0.0859 - val_accuracy: 0.9767
Epoch 4/5
540/540 - 25s - loss: 0.0729 - accuracy: 0.9781 - val_loss: 0.0710 - val_accuracy: 0.9817
Epoch 5/5
540/540 - 23s - loss: 0.0603 - accuracy: 0.9819 - val_loss: 0.0632 - val_accuracy: 0.9822


### Test the model

In [32]:
test_loss, test_accuracy = model.evaluate(test_data)

      1/Unknown - 4s 4s/step - loss: 0.0874 - accuracy: 0.97 - 4s 4s/step - loss: 0.0874 - accuracy: 0.9724

In [33]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.09. Test accuracy: 97.24%
